In [1]:
import numpy as np 
import argparse
import pandas as pd 
import netCDF4



def get_parent_flow(usgs_id, year):
    #Get the list of parents from the csv 
    a = usgs_p.loc[usgs_id, 'from_usgs']
    parents = np.array(a[1:-1].split(','))
    
    #Get the flow data from the parents
    flow_parents = []
    existing_childs = []
    for parent in parents:
        try:
            qo = np.load('/mnt/y/flow/USGS_observed/%d/%d_usgs_%s.npz' % (year, year,parent))
            uxt_u = np.unique(qo['data']['uxt'])
            #Converts the observed flow to match the nwc forecast
            flow_o = np.ones(d.dimensions['issue_time'].size)*-999
            shared1 = np.argwhere(np.in1d(uxt, uxt_u)).T[0]
            shared2 = np.argwhere(np.in1d(uxt_u, uxt)).T[0]
            flow_o[shared1] = qo['data']['val'][shared2]*0.028
            flow_o[flow_o <= 0] = np.nan
            flow_parents.append(flow_o)
            existing_childs.append(parent)
        except:
            pass
    flow_parents = np.array(flow_parents)
    parents_sum = flow_parents.sum(axis = 0)
    
    qo = np.load('/mnt/y/flow/USGS_observed/%d/%d_usgs_%s.npz' % (year, year,usgs_id))
    uxt_u = np.unique(qo['data']['uxt'])
    #Converts the observed flow to match the nwc forecast
    flow_o = np.ones(d.dimensions['issue_time'].size)*-999
    shared1 = np.argwhere(np.in1d(uxt, uxt_u)).T[0]
    shared2 = np.argwhere(np.in1d(uxt_u, uxt)).T[0]
    flow_o[shared1] = qo['data']['val'][shared2]*0.028
    flow_o[flow_o <= 0] = np.nan
    
    flow_scaled = np.nanmean(flow_o) * (parents_sum / np.nanmean(parents_sum))
    
    #Computes the errors of using the persistence 
    Ep = []
    Es = []
    for lead in range(1,19):
        a = 100*(parents_sum-np.roll(flow_o,lead))/np.roll(flow_o,lead)
        Ep.append(a.astype(int))
        a = 100*(flow_scaled-np.roll(flow_o,lead))/np.roll(flow_o,lead)
        Es.append(a.astype(int))
    
    results = {
        'childs': existing_childs,
        'flow': flow_o,
        'flow_childs':flow_parents,
        'sum_childs':parents_sum,
        'scaled_childs': flow_scaled,
        'er_chi': Ep,
        'er_sca': Es
    }
    
    return results

year = 2022

usgs_p = pd.read_csv('../data/usgs_parents_v2.csv', usecols=[1,4],dtype = {'to_usgs':str,'from_usgs':'str'}, index_col=0)
d = netCDF4.Dataset('/mnt/y/flow/NWC_forecast/%d_nwc_short.nc' % year)
usgs = d['usgs_id'][:]
uxt = d['issue_time'][:]


In [7]:
real_childs = {}
for pos, gauge in enumerate(usgs_p.index.values[:5]):
    try:
        childs = get_parent_flow(gauge, year)
#         error_simple[pos,:,:] = childs['er_chi']
#         error_scaled[pos,:,:] = childs['er_sca']
#         per_simple[pos,:] = childs['sum_childs']*100
#         per_scaled[pos,:] = childs['scaled_childs']*100
        real_childs.update({gauge:childs['childs']})
        real_childs[gauge] = '{' +','.join(real_childs[gauge]) + '}'
        print(pos, usgs[pos], 0)
    except:
        print(pos, usgs[pos], 1)


0 01030350 0
1 01030500 1
2 06741510 0
3 402114105350101 0
4 272524080221800 0
